# Text2GART

In [1]:
%load_ext autoreload
%autoreload 2
import os, os.path as osp
from solver import TGFitter
import torch
from viz_utils import viz_human_all

In [ ]:
from lib_guidance.mvdream.mvdream_guidance import MVDream

# guidance = MVDream(torch.device("cuda"), fp16=True, n_view=4, t_range=[0.02, 0.98]) # seems reduce 20% time
guidance = MVDream(torch.device("cuda"), fp16=False, n_view=4, t_range=[0.02, 0.98])

In [3]:
texts = [
    "A yellow CyberPunk robot, silver skeleton",
    "A frog character from a game",
    # "A red Gundam Zaku, with a jet backpack",
    # "A doctor in green surgical uniform",
    # "A policeman in blue uniform",
    # "A white humanoid robot from 23 century with red eyes",
    # "A silver robot with single red eye like hal9000",
    # "An energetic computer science professor, wit blue t-shirt, black jeans",
    # "A knight with a sword on his hand",
    # "Skywalker",
    # "A pink cute ellipsoid robot",
    # "Spiderman with batman suit",
    # "A walking Banana",
    # "A dinosaur",
    # "A dog",
]

In [ ]:
profile_fn = "./profiles/text2gart/gen.yaml"
base_name = "text2gart"
# # or you could try this, adding camera location near hands, but seems not that helpful
# profile_fn = "./profiles/text2gart/gen_hand.yaml"
# base_name = "text2gart_hand_enhance"

os.makedirs(f"./logs/{base_name}", exist_ok=True)

for txt in texts:
    torch.cuda.empty_cache()
    print(txt)
    guidance.set_text_embeds(
        [txt + "; smooth texture, clean image, single subject image, black background"],
        [
            "ugly, bad anatomy, blurry, pixelated obscure, unnatural colors, poor lighting, dull, and unclear, lowres, low quality, artifacts, duplicate, morbid, mutilated, poorly drawn face, dehydrated, bad proportions"
        ],  # remove the cropped, deformed
    )

    log_id = len(os.listdir(f"./logs/{base_name}")) + 1
    log_dir = f"./logs/{base_name}/{log_id}_{txt}"

    solver = TGFitter(
        log_dir=log_dir,
        profile_fn=profile_fn,
        mode="human",
        template_model_path="./data/smpl_model/SMPL_MALE.pkl",
        device=torch.device("cuda:0"),
        NO_TB=True,
    )

    data_provider = solver.prepare_fake_data(
        mode="amass",
        pose_dirs=["./data/amass/BMLrub/"],
        da_pose_prob=0.0,
    )

    _, _ = solver.run(fake_data_provider=data_provider, guidance=guidance)
    viz_human_all(solver, None, training_skip=1)